In [76]:
import requests
import time
from bs4 import BeautifulSoup
import json

In [77]:
def extract_info(obj):
    TEXT_LABLE = "article-text response-page__text markup-inside-small markup-inside-small--bullet"
    RATING_LABLE = "color-gray-gray font-size-small margin-bottom-default margin-top-small"
    CITY_LABLE = "response-page__bank-meta font-size-medium color-gray-burn"
    BANK_LABLE = "header-h2 display-inline margin-right-x-small"
    TITLE_LABLE = "header-h0 response-page__title"
    SENDER_LABLE = "link-with-icon__text color-gray-gray padding-right-xx-small"
    POST_DATE_LABLE = "dtreviewed"
    
    text = obj.find("div", attrs={"class": TEXT_LABLE}).text.strip().replace('\xa0', '')
    bank = obj.find("div", attrs={"class": BANK_LABLE}).text.strip()
    title = obj.find("h0", attrs={"class": TITLE_LABLE}).text.strip()
    city = obj.find("div", attrs={"class": CITY_LABLE}).text.split(', ')
    rating = obj.find("div", attrs={"class": RATING_LABLE}).text.strip()
    sender = obj.find("span", attrs={"class": SENDER_LABLE}).text
    post_date = obj.find("time", attrs={"itemprop": POST_DATE_LABLE}).text
    
    return {
        'bank': bank,
        'city': city[1].split('.')[-1].strip() if len(city) > 1 else None,
        'title': title,
        'text': text,
        'rating': None if 'БЕЗ ОЦЕНКИ' in rating else int(rating.split()[1]),
        'sender': sender,
        'post_date': post_date.strip()
    }

In [80]:
%%time

url = 'http://www.banki.ru/services/responses/bank/response/'

result = list()
start_index = 10000008

#parsing of 1000 reviews
success_reviews = 0
count = 0
while success_reviews < 1000:
    response = requests.get(url + str(start_index + count))
    time.sleep(2)
    count += 1
    if count % 10 == 0:
        print('iteration - {}, successful - {}'.format(count, success_reviews))
    
    if response.status_code != 200:
        print('Error {}, iteration - {}'.format(response.status_code, count))
        continue
    
    success_reviews += 1
    
    bs = BeautifulSoup(response.text, 'html5lib')
    result.append(extract_info(bs))

iteration - 10, successful - 9
iteration - 20, successful - 19
iteration - 30, successful - 29
Error 404, iteration - 32
Error 404, iteration - 34
Error 404, iteration - 35
iteration - 40, successful - 36
Error 404, iteration - 40
Error 404, iteration - 44
iteration - 50, successful - 44
iteration - 60, successful - 54
Error 404, iteration - 65
iteration - 70, successful - 63
iteration - 80, successful - 73
iteration - 90, successful - 83
Error 404, iteration - 97
Error 404, iteration - 99
iteration - 100, successful - 91
Error 404, iteration - 103
iteration - 110, successful - 100
Error 404, iteration - 110
iteration - 120, successful - 109
Error 404, iteration - 124
iteration - 130, successful - 118
Error 404, iteration - 133
Error 404, iteration - 134
Error 404, iteration - 135
iteration - 140, successful - 125
Error 404, iteration - 148
iteration - 150, successful - 134
Error 404, iteration - 158
iteration - 160, successful - 143
iteration - 170, successful - 153
iteration - 180, s

ConnectionError: HTTPConnectionPool(host='www.banki.ru', port=80): Max retries exceeded with url: /services/responses/bank/response/10000231 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000023836CF6240>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера',))

In [81]:
with open('HW06_Data_Parshin.json', 'w') as f:
    json.dump(result, f)